In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import LabelEncoder
from collections import Counter

from keras import layers, models, optimizers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
folder_path = "./images_coronavirus_6class"

X = []
y = []

image_filenames = os.listdir(folder_path)

for filename in image_filenames:
    image_class = filename.split('_')[-1]
    image = cv2.imread(os.path.join(folder_path, filename))
    image = cv2.resize(image, (32, 32))
    X.append(image)
    y.append(int(image_class.split('.')[0]))

X = np.array(X)
y = np.array(y)

In [3]:
print(X.shape, y.shape)
print(np.unique(y, return_counts=True))

(955, 32, 32, 3) (955,)
(array([0, 1, 2, 3, 4, 5]), array([186, 100, 200,  69, 200, 200], dtype=int64))


In [4]:
permutation_index = np.random.permutation(len(X))

X = X[permutation_index]
y = y[permutation_index]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [6]:
def create_cnn_model():
    model = models.Sequential()
    
    model.add(layers.InputLayer(input_shape=(32, 32, 3)))
    model.add(layers.Rescaling(1./255))  
    
    # Convolutional layers
    model.add(Conv2D(16, (5, 5), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    
    model.add(Dropout(0.2))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(6, activation='softmax'))  
    
    return model

In [7]:
model = create_cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 16)        1216      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 32)          4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 32)          9248      
                                                                 
 dropout (Dropout)           (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                        

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(
        X_test,
        y_test
    )
)

Epoch 1/20
26/26 [==============================] - 2s 34ms/step - loss: 1.6588 - accuracy: 0.2602 - val_loss: 1.5837 - val_accuracy: 0.5139
Epoch 2/20
26/26 [==============================] - 0s 16ms/step - loss: 1.2406 - accuracy: 0.4587 - val_loss: 1.1214 - val_accuracy: 0.5764
Epoch 3/20
26/26 [==============================] - 0s 16ms/step - loss: 1.0292 - accuracy: 0.5438 - val_loss: 1.0127 - val_accuracy: 0.7778
Epoch 4/20
26/26 [==============================] - 0s 14ms/step - loss: 0.8720 - accuracy: 0.6732 - val_loss: 0.8490 - val_accuracy: 0.8472
Epoch 5/20
26/26 [==============================] - 0s 17ms/step - loss: 0.7686 - accuracy: 0.7176 - val_loss: 0.7302 - val_accuracy: 0.8958
Epoch 6/20
26/26 [==============================] - 0s 13ms/step - loss: 0.6715 - accuracy: 0.7448 - val_loss: 0.6629 - val_accuracy: 0.8750
Epoch 7/20
26/26 [==============================] - 0s 14ms/step - loss: 0.5753 - accuracy: 0.7744 - val_loss: 0.6002 - val_accuracy: 0.7639
Epoch 8/20
26

In [10]:
scores = model.evaluate(X_test, y_test, verbose=1)
print(f'test accuracy is {scores[1]}')

5/5 [==============================] - 0s 6ms/step - loss: 0.2223 - accuracy: 0.9097
test accuracy is 0.9097222089767456


In [13]:
!mkdir -p saved_model_coronavirus_classification
model.save('./saved_model_coronavirus_classification/coronavirus_6_class_classification.keras')

A subdirectory or file -p already exists.
Error occurred while processing: -p.
